In [1]:
from dataclasses import dataclass
import sys
sys.path.append('/data2/yanjiahuan/research/testProject202301_underGoing')
from data_utils import init_data_handler

/data2/yanjiahuan/.conda/envs/torch190/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
@dataclass
class DataArgs:
    dataset: str
data_args = DataArgs('pubmed_multilabel')
Xs, id2label = init_data_handler(None, data_args,return_raw=True)

Loading dataset: PUBMED_MULTILABEL
using cache in: /data2/yanjiahuan/research/testProject202301_underGoing/data/pubmed_multilabel


In [19]:
Xs['test']['labels'].shape

(10000, 100)

In [7]:
parent_labels = sorted(list(set([v.split('.')[0] for v in id2label.values()])))
child_labels = list(id2label.values())

In [8]:
valuedict = {i: v for i, v in enumerate(parent_labels + child_labels)}
v2i = {v:k for k,v in valuedict.items()}

In [9]:
import json
with open('/data2/yanjiahuan/research/testProject202301_underGoing/data/mimic3-top50/all_label2text.json', 'r') as f:
    label2text = json.load(f)

In [10]:
new_valuedict = {i: label2text[v] for i, v in valuedict.items()}
# new_valuedict = valuedict

In [15]:
new_valuedict

{0: 'Musculoskeletal System',
 1: 'Cardiovascular System',
 2: 'Nervous System',
 3: 'Cells',
 4: 'Hemic and Immune Systems',
 5: 'Eukaryota',
 6: 'Bacteria',
 7: 'Infections',
 8: 'Neoplasms',
 9: 'Digestive System Diseases',
 10: 'Respiratory Tract Diseases',
 11: 'Nervous System Diseases',
 12: 'Female Genital Diseases and Pregnancy Complications',
 13: 'Cardiovascular Diseases',
 14: 'Skin and Connective Tissue Diseases',
 15: 'Nutritional and Metabolic Diseases',
 16: 'Pathological Conditions, Signs and Symptoms',
 17: 'Inorganic Chemicals',
 18: 'Organic Chemicals',
 19: 'Heterocyclic Compounds',
 20: 'Polycyclic Compounds',
 21: 'Macromolecular Substances',
 22: 'Hormones, Hormone Substitutes, and Hormone Antagonists',
 23: 'Enzymes and Coenzymes',
 24: 'Amino Acids, Peptides, and Proteins',
 25: 'Nucleic Acids, Nucleotides, and Nucleosides',
 26: 'Biological Factors',
 27: 'Chemical Actions and Uses',
 28: 'Diagnosis',
 29: 'Therapeutics',
 30: 'Investigative Techniques',
 31: 

In [27]:
import torch
torch.save(new_valuedict, "./data/pubmed_multilabel2/value_dict.pt")

In [28]:
slot2value = {}
for v, i in v2i.items():
    if '.' not in v and i not in slot2value:
        slot2value[i] = set()
    else:
        parent = v.split('.')[0]
        parent_i = v2i[parent]
        if parent_i not in slot2value:
            slot2value[parent_i] = set()
        slot2value[parent_i].add(i)

In [29]:
from collections import defaultdict
slot2value = defaultdict(set, slot2value)

In [30]:
torch.save(slot2value, "./data/pubmed_multilabel2/slot.pt")

In [31]:
value2id = {v: k for k, v in valuedict.items()}

In [ ]:
value2id

In [33]:
from tqdm import tqdm
import numpy as np

def build_htc_label(label):
    label_ids = []
    if isinstance(label, np.ndarray):
        label_ids = np.where(label == 1)[0].tolist()
    elif isinstance(label, np.integer):
        label_ids = [label]
    htc_labels = []
    label_names = []
    for i in label_ids:
        v = id2label[i]
        label_names.append(v)
        p = v.split('.')[0]
        if p not in htc_labels:
            htc_labels.append(p)
    htc_labels += label_names
    htc_ids = [value2id[v] for v in htc_labels]
    return htc_ids

def write_data(Xs, spl, dataset):
    print("writing dataset: ", dataset, " ", spl)
    with open(f'./data/{dataset}/{dataset}_{spl}.json', 'w') as f:
        lines = [json.dumps(X, ensure_ascii=False) + "\n" for X in Xs]
        f.writelines(lines)
    print('done')

for spl in Xs:
    new_Xs = []
    texts, labels = Xs[spl]['texts'], Xs[spl]['labels']
    for text, label in tqdm(zip(texts, labels), desc=spl):
        new_X = {
            "token": text,
            "label": build_htc_label(label)}
        new_Xs.append(new_X)
    write_data(new_Xs, spl, 'pubmed_multilabel2')

train: 32000it [00:00, 79229.61it/s]


writing dataset:  pubmed_multilabel2   train
done


dev: 8000it [00:00, 122324.82it/s]

writing dataset:  pubmed_multilabel2   dev


done


test: 10000it [00:00, 124685.75it/s]

writing dataset:  pubmed_multilabel2   test


done


In [137]:
import torch
slot2value = torch.load('/data2/yanjiahuan/research/testProject202301_underGoing/baselines/HPT/data/dermatology/slot.pt')
valuedict = torch.load('/data2/yanjiahuan/research/testProject202301_underGoing/baselines/HPT/data/dermatology/value_dict.pt')

In [138]:
slot2value

defaultdict(set,
            {0: {41, 42},
             1: {43},
             2: {44},
             3: {45, 46, 47, 48},
             4: {49},
             5: {50},
             6: {51, 52, 53, 54},
             7: {55},
             8: {56},
             9: {57},
             10: {58},
             11: {59},
             12: {60},
             13: {61},
             14: {62, 63},
             15: {64},
             16: {65},
             17: {66},
             18: {67, 68},
             19: {69},
             20: {70, 71, 72, 73, 74},
             21: {75, 76},
             22: {77},
             23: {78, 79, 80},
             24: {81},
             25: {82},
             26: {83},
             27: {84},
             28: {85},
             29: {86},
             30: {87},
             31: {88},
             32: {89},
             33: {90},
             34: {91},
             35: {92},
             36: {93},
             37: {94},
             38: {95},
             39: {96},
         

In [27]:
torch.load('/data2/yanjiahuan/research/testProject202301_underGoing/baselines/HPT/data/WebOfScience/slot.pt')

defaultdict(set,
            {0: {7,
              12,
              23,
              28,
              37,
              41,
              46,
              49,
              50,
              57,
              91,
              99,
              101,
              110,
              111,
              125,
              140},
             1: {8,
              11,
              16,
              19,
              21,
              22,
              27,
              35,
              36,
              38,
              39,
              40,
              43,
              44,
              47,
              52,
              53,
              54,
              63,
              65,
              68,
              69,
              70,
              72,
              73,
              74,
              75,
              76,
              78,
              80,
              85,
              89,
              106,
              107,
              108,
              109,
              1